In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import json
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde
import numpy as np
import time

import nomad.io.base as loader
import nomad.city_gen as cg
import nomad.traj_gen as tg
from nomad.traj_gen import Agent, Population
from nomad.city_gen import City
from nomad.map_utils import blocks_to_mercator_gdf

import nomad.data as data_folder
from pathlib import Path
data_dir = Path(data_folder.__file__).parent
path = data_dir / "garden_city.gpkg"

In [11]:
traj_df = loader.from_file('sparse_traj_2/part-0.parquet', format='parquet')
diaries_df = loader.from_file('diaries_2/part-0.parquet', format='parquet')
destinations = pd.read_csv('exp_2_destinations_unbalanced.csv')

In [ ]:
start_time = time.time()
samples = traj_df[['x', 'y']].values
x, y = samples[:,0], samples[:,1]

kde = gaussian_kde(samples.T, bw_method=0.5)

# Build grid over the data range
xmin, xmax = X[:, 0].min(), X[:, 0].max()
ymin, ymax = X[:, 1].min(), X[:, 1].max()

xx, yy = np.meshgrid(
    np.linspace(xmin, xmax, 100),
    np.linspace(ymin, ymax, 100)
)

grid = np.vstack([xx.ravel(), yy.ravel()])

# Evaluate log density
Z = np.reshape(kde(grid), xx.shape)
dens = Z

print(time.time() - start_time)

In [ ]:
# --------------------- Plotting ---------------------
plt.style.use('dark_background')
fig = plt.figure(figsize=(14, 6))

# Left: 2D contours
ax1 = fig.add_subplot(121)
levels = np.linspace(0, Z.max(), 15)[2:]  # clean levels
ax1.contourf(xx, yy, dens, levels=levels, cmap='inferno', alpha=0.95)
ax1.contour(xx, yy, dens, levels=levels[::2], colors='white', linewidths=0.7, alpha=0.8)
ax1.set_xlabel('X', fontsize=14)
ax1.set_ylabel('Y', fontsize=14)
ax1.set_aspect('equal')

# Right: 3D surface
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_surface(xx, yy, dens, cmap='inferno', linewidth=0, 
                        antialiased=False, alpha=0.98, rcount=300, ccount=300)

ax2.set_xlabel('X', labelpad=10)
ax2.set_ylabel('Y', labelpad=10)
ax2.set_zlabel('Density', labelpad=10)

ax2.view_init(elev=35, azim=-65)

# Colorbar
m = plt.cm.ScalarMappable(cmap='inferno')
m.set_array(Z)
fig.colorbar(m, ax=ax2, shrink=0.7, aspect=20, pad=0.02)

plt.tight_layout()
plt.savefig('test_plot.png')
print("Code executed successfully. Plot saved as test_plot.png")
print("Z shape:", dens.shape)
print("Max Z:", np.max(dens))
print("Sample data shape:", samples.shape)